In [1]:
# import

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from datetime import date
import time
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
# hyperlinks

interiaHyperlink = 'https://wydarzenia.interia.pl'
onetHyperlink = 'https://www.onet.pl/'

In [3]:
# loading interia links on page

s = Service('chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get("https://wydarzenia.interia.pl/")
button_div_xpath = "/html/body/div[7]/div/div/section/div"

time.sleep(1)
while True:
    try:       
        show_more_button_div = driver.find_element(By.XPATH, button_div_xpath)
        s = BeautifulSoup(show_more_button_div.get_attribute('innerHTML'))
        s2 = s.find('button')
        #print(str(s2['class']))
        
        if "hidden" in str(s2['class']):
            print("All loaded")
            break        
        
        show_more_button = driver.find_element(By.XPATH, button_div_xpath + "/button")
        driver.execute_script("arguments[0].click();", show_more_button)
        print("Show more button clicked")
        time.sleep(0.5)
    except:
        print("Error: No \"Show more\" button detected")
        break
        
html_text = driver.page_source
driver.close()


Show more button clicked
Show more button clicked
Show more button clicked
Show more button clicked
Show more button clicked
All loaded


In [12]:
# get interia links

soup = BeautifulSoup(html_text)
news_soup = soup.find_all('a', class_ = 'tile-magazine-title-url')
#print("Number of news: " + str(len(news_soup)))

d = {'Title': [], 'Address':[]}
news_DF = pd.DataFrame(data=d)

for m in news_soup:  
    x = pd.DataFrame(data={'Title': [m['title']], 'Address': [interiaHyperlink + m['href']]})
    news_DF = pd.concat([news_DF, x], ignore_index = True)

news_DF.to_csv( 'News '+ str(date.today()) + '.csv', sep=';')

https://wydarzenia.interia.pl/zagranica/news-japonia-otrzymal-ogromne-pieniadze-z-rzadowego-funduszu-prze,nId,6030844


In [55]:
print(news_DF['Address'].iloc[10])

a = news_DF['Address'].iloc[10]

html_text = requests.get(a)
soup = BeautifulSoup(html_text.text)
article_container = soup.find('div', class_ = 'article-container')

article_body = article_container.find_all('p')

#remove block of text - useless
article_body.pop(1)

#remove advertisment
article_body.pop(len(article_body)-1)

t = ''

for i in article_body:
    #print(i.text)
    t = t + ' ' + i.text


print(t)

https://wydarzenia.interia.pl/zagranica/news-japonia-otrzymal-ogromne-pieniadze-z-rzadowego-funduszu-prze,nId,6030844
 46,3 mln jenów (ponad 1,5 mln złotych) z dnia na dzień pojawiło się na koncie 24-letniego Japończyka. To efekt pomyłki urzędników - cała kwota pochodziła z lokalnego funduszu pomocy pocovidowej, przeznaczonego dla osób w trudnej sytuacji życiowej. Pieniędzy nie można odzyskać, bo 24-latek roztrwonił je w dwa tygodnie. Pomyłkę popełnili urzędnicy z miasta Abu w prefekturze Yamaguchi w południowej części Japonii. 463 gospodarstwa domowe, które najmocniej odczuły wpływ pandemii COVID-19 miały otrzymać po 100 tys. jenów (ok. 3,4 tys. złotych) w ramach rządowego programu pomocy. Jednak 8 kwietnia cała kwota - 46,3 mln jenów - została przelana na konto 24-latka. Zobacz też: Nie żyje prezydent Zjednoczonych Emiratów Arabskich Chalifa ibn Zajid Al Nahajjan. Miał 73 lata Mężczyzna nie tylko nie zgłosił sprawy, ale zaczął błyskawicznie wydawać pieniądze z funduszu. Przez dwa tyg

In [56]:
import morfeusz2


x = morfeusz2.Morfeusz()
a = t
result = x.analyse(a)

print(a)
for r in result:
    print(r)

 46,3 mln jenów (ponad 1,5 mln złotych) z dnia na dzień pojawiło się na koncie 24-letniego Japończyka. To efekt pomyłki urzędników - cała kwota pochodziła z lokalnego funduszu pomocy pocovidowej, przeznaczonego dla osób w trudnej sytuacji życiowej. Pieniędzy nie można odzyskać, bo 24-latek roztrwonił je w dwa tygodnie. Pomyłkę popełnili urzędnicy z miasta Abu w prefekturze Yamaguchi w południowej części Japonii. 463 gospodarstwa domowe, które najmocniej odczuły wpływ pandemii COVID-19 miały otrzymać po 100 tys. jenów (ok. 3,4 tys. złotych) w ramach rządowego programu pomocy. Jednak 8 kwietnia cała kwota - 46,3 mln jenów - została przelana na konto 24-latka. Zobacz też: Nie żyje prezydent Zjednoczonych Emiratów Arabskich Chalifa ibn Zajid Al Nahajjan. Miał 73 lata Mężczyzna nie tylko nie zgłosił sprawy, ale zaczął błyskawicznie wydawać pieniądze z funduszu. Przez dwa tygodnie wypłacał średnio 600 tys. jenów dziennie (ponad 20 tys. złotych). Majątek bezskutecznie próbował powiększyć graj